In [ ]:
from bokeh.io import output_notebook
# This tells Bokeh to display output in the notebook versus opening a browser window
output_notebook()

In [ ]:
# toy problem with local detuning
from bloqade import start
import time

# Define relevant parameters
durations = [0.8, "sweep_time", 0.8]  # (us)
register = start.add_position([(0,0),(5,0),(10,0)])  # (um)
omega_max=15.7  # (rad/us)
delta_glob_max=124.  # (rad/us)
delta_loc_max=62.  # (rad/us)

# add local detuning
atomLab=[0,1,2]  # atom names labeled by their placement order. Counting from 0?
atomScale=[0.7,0.5,0.3]  # (range [0,1])  those are  modulations for the common local detune

prog = (
    register
    .rydberg.rabi.amplitude.uniform.piecewise_linear(
        durations, values=[0.0, "rabi_drive", "rabi_drive", 0.0])
    .rydberg.detuning.uniform.piecewise_linear(
        durations, values=["detune_min","detune_min","detune_max","detune_max"])
    .location(atomLab,atomScale).piecewise_linear(
        durations, values=[0.,0.,"detune_local","detune_local"])
)

job = prog.assign(sweep_time=1.2,rabi_drive=omega_max,
            detune_min=-delta_glob_max, detune_max=delta_glob_max, detune_local=delta_loc_max)
#print(program)
t0=time.time()
emu_batch = job.braket.local_emulator().run(100)
print('run elaT=%.1f sec'%(time.time()-t0))
report = emu_batch.report()
counts=report.counts   # does it remove 'e' from pre-measure and is '1'=rydberg?
print('Counts:',counts)


In [ ]:
print(prog)

In [ ]:
emu_batch.source.show()